<a href="https://colab.research.google.com/github/SmartPracticeschool/SBSPS-Challenge-2700-Twitter-Sentiment-Analysis-Extraction-for-COVID-19/blob/master/Notebooks/Data_Creation_%26_Cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


#Modules Installation

In [ ]:
!pip install -q twarc
!pip install -q jsonlines

#Import Modules

In [ ]:
import os,zipfile,glob
import pandas as pd
#import twarc

Unzip Files

In [ ]:

dir_name = '/content/drive/My Drive/IBM_Hackathon_2020/Zip_Files'
extension = ".zip"

os.chdir(dir_name) # change directory from working dir to dir with files

for item in os.listdir(dir_name): # loop through items in dir
    if item.endswith(extension): # check for ".zip" extension
        file_name = os.path.abspath(item) # get full path of files
        zip_ref = zipfile.ZipFile(file_name) # create zipfile object
        zip_ref.extractall('/content/drive/My Drive/IBM_Hackathon_2020/Tweet_Csv_File') # extract file to dir
        zip_ref.close() # close file
        os.remove(file_name) # delete zipped file

#Total IEEE Dataset 




In [ ]:
path = "/content/drive/My Drive/IBM_Hackathon_2020/Tweet_Csv_File/*.csv"
csv_list = glob.glob(path) # collecting all files  same path 
print(len(csv_list))

In [ ]:
data = pd.DataFrame()
for f in csv_list:
  data1 = pd.read_csv(f,header=None)#reading the csv file
  data = pd.concat([data,data1],ignore_index=True)#concating the two data frames
  data.reset_index(drop=True,inplace=True) #resetting the indexes

In [ ]:
data.rename({0:'tweetID',1:'sentiment_score'},axis=1,inplace=True)#renaming the indexes

In [ ]:
data.to_csv('/content/drive/My Drive/IBM_Hackathon_2020/Tweet_Dataset/data.csv',index=False)#converting the dataframe to CSV

In [ ]:
for f in csv_list:
  df = pd.read_csv(f,header=None) #reading the CSV file
  df = df[0] #Only taking the Tweet Id's from the dataset
  base = os.path.basename(f) #returning the name of the file
  path = '/content/drive/My Drive/IBM_Hackathon_2020/Tweets_ID/'+base
  df.to_csv(path,index=False) # converting the dataframe to CSV 

# Hydrating Tweets


In [ ]:
# Insert API Keys here { run : "auto"}

# These keys are received after applying for a twitter developer account

consumer_key = "rm2bLDjA2BzljoA0GomL5o6W7"
consumer_secret = "xiFBG4VKWPuQts1v3uqAesllpDp36y44YkFnzBtezSbSYW9dBV"
access_token = "935519854064418816-sOBxmFMaDygAx3FQXRBjH0drpZ2OXpB"
access_token_secret = "GbOTefzapdet9vpmR3H9OBRuJNJNs1cI4Adh5HrkIYPJz"

t = twarc.Twarc(consumer_key, consumer_secret, access_token, access_token_secret) #Initializing Twarc

In [ ]:
import jsonlines, json

data = pd.read_csv('/content/drive/My Drive/IBM_Hackathon_2020/Geo_Location_Data_set/april28-june18.csv',header=None) #Loading IEEE Geodata
data = data[0] #Taking only Tweet IDs for Hydration
ids = data.values.tolist() #Getting list of tweet ids from pandas DataFrame
hydrated_tweets = [] #Creating empty list
ids_to_hydrate = set(ids) #Creating ids_to_hydrate list

# Now, use twarc and start hydrating
for tweet in t.hydrate(ids_to_hydrate): #Initializing Hydrate Iterator with twarc
  if tweet['place'] != None: #Checking for place value
    if tweet['place']['country'] == 'India': #Checking if Country is India
      hydrated_tweets.append(tweet) #Appending Obtained tweet to hydrated_tweets list

In [ ]:
output_filename = "/content/drive/My Drive/IBM_Hackathon_2020/Geo_Location_Data_set/output.csv" #output file path

In [ ]:
# Convert jsonl to csv
import csv, jsonlines

# These are the column name that are selected to be stored in the csv
keyset = ["created_at", "id", "id_str", "full_text", "source", "truncated", "in_reply_to_status_id",
          "in_reply_to_status_id_str", "in_reply_to_user_id", "in_reply_to_user_id_str", 
          "in_reply_to_screen_name", "user", "coordinates", "place", "quoted_status_id",
          "quoted_status_id_str", "is_quote_status", "quoted_status", "retweeted_status", 
          "quote_count", "reply_count", "retweet_count", "favorite_count", "entities", 
          "extended_entities", "favorited", "retweeted", "possibly_sensitive", "filter_level", 
          "lang", "matching_rules", "current_user_retweet", "scopes", "withheld_copyright", 
          "withheld_in_countries", "withheld_scope", "geo", "contributors", "display_text_range",
          "quoted_status_permalink"]

# Writes them out (Saving output CSV file with the Indian COVID-19 Tweets)
with  open(output_filename, "w+") as output_file:
    d = csv.DictWriter(output_file, keyset)
    d.writeheader()
    d.writerows(hydrated_tweets)

In [ ]:
import pandas as pd

data = pd.read_csv(output_filename)# reading the file
data1 = pd.read_csv('/content/drive/My Drive/IBM_Hackathon_2020/Geo_Location_Data_set/april28-june18.csv',header=None)
data1.rename({0:'id',1:'Sentiment'},inplace=True,axis=1)#renaming the columns
result = pd.merge(data,data1,on='id')#merging the two dataframes
result.drop_duplicates(subset ="id", keep = False, inplace = True)#removing duplicates
result.to_csv('/content/drive/My Drive/IBM_Hackathon_2020/Tweet_Dataset/India_Dataset.csv',index=False)# Converting to CSV

#*Sentiment Addition*

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv("/content/drive/My Drive/hydrated_corona_tweets_01")
df = df[['id','retweet_count','lang','text']]#selection of Features
data = df[df["lang"]=="en"]#Extracting the tweets from dataframe whose language is English

In [ ]:
tweet_ids_file = "/content/drive/My Drive/IBM_Hackathon_2020/Tweet_Csv_File/corona_tweets_01.csv" #Reference Original CSV Path

In [ ]:
data = pd.read_csv(input_filename)# reading the file
data.rename({'text':'full_text'},inplace=True,axis=1)
dataset = data[['id','full_text','retweet_count']]
df = pd.read_csv(tweet_ids_file,header=None)
df.rename({0:'id',1:'Sentiment'},inplace=True,axis=1)#renaming the columns
output = pd.merge(dataset,df,on='id')#merging (inner) the two dataframes
output.drop_duplicates(subset ="id", keep = False, inplace = True)#removing duplicates
output.to_csv('/content/drive/My Drive/IBM_Hackathon_2020/Tweet_Dataset/Dataset.csv',index=False)# Converting to CSV

In [ ]:
# Function to Convert the IEEE Sentiment score to Sentiment Text namely positive, negative, neutral
def func(x):
    if x < 0:
        return "negative"
    elif x == 0:
        return "neutral"
    else:
        return "positive"

In [ ]:
df1 = pd.read_csv("/content/drive/My Drive/IBM_Hackathon_2020/Tweet_Dataset/India_Dataset.csv")# reading the file
y= df1.Sentiment
X= df1.drop('Sentiment',axis=1)
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.75,random_state=111)#Spliting the dataset 
df2 = pd.concat([X_train,y_train],axis=1)#concating the two datasets
df3 = pd.concat([X_test,y_test],axis=1)#concating the two datasets
df3.reset_index(drop=True,inplace=True)#resetting the indexes
df3['Sentiment'] = df3['Sentiment'].apply(lambda x : func(x))#converting sentiment score to sentiment text
df3.to_csv('/content/drive/My Drive/IBM_Hackathon_2020/Final_Datasets/Test_India.csv',index=False)# Converting to CSV

In [ ]:
data = pd.read_csv('/content/drive/My Drive/IBM_Hackathon_2020/Tweet_Dataset/Dataset.csv')# reading the file
df4=data.sample(n=58944,random_state=123)# sampling the dataset
df5=pd.concat([df2,df4],ignore_index=True)#Concating the two datasets 
df5.reset_index(drop=True,inplace=True)# restting the indexes
df5['Sentiment'] = df5['Sentiment'].apply(lambda x : func(x))#converting sentiment score to sentiment text
df5.to_csv("/content/drive/My Drive/IBM_Hackathon_2020/Final_Datasets/Data.csv",index=False)# Converting to CSV

# Splitting Final Data to Train and Test sets for DL Model


In [ ]:
data = pd.read_csv('/content/drive/My Drive/IBM_Hackathon_2020/Final_Datasets/Data.csv')# reading the file
y = data.Sentiment
X = data.drop('Sentiment',axis=1)
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25,random_state=123)
df1 = pd.concat([X_train,y_train],axis=1)#concating the two datasets
df1.reset_index(drop=True,inplace=True)#resetting the indexes
df1.to_csv('/content/drive/My Drive/IBM_Hackathon_2020/Final_Datasets/Train_Data.csv',index=False) # Saving Training/Validation data for DL Model
df2 = pd.concat([X_test,y_test],axis=1)#concating the two datasets
df2.reset_index(drop=True,inplace=True)#resetting the indexes
df2.to_csv('/content/drive/My Drive/IBM_Hackathon_2020/Final_Datasets/Test_Data.csv',index=False)# Saving Test data for DL Model

#*Data Cleaning*

In [ ]:
#Defining functions for Cleaning and Normalization of Data
import re
import string
def replace_url(string): # cleaning of URL
    text = re.sub(r'http\S+', 'LINK', string)
    return text


def replace_email(text):#Cleaning of Email related text
    line = re.sub(r'[\w\.-]+@[\w\.-]+','MAIL',str(text))
    return "".join(line)

def rep(text):#cleaning of non standard words
    grp = text.group(0)
    if len(grp) > 3:
        return grp[0:2]
    else:
        return grp# can change the value here on repetition
def unique_char(rep,sentence):
    convert = re.sub(r'(\w)\1+', rep, sentence) 
    return convert

def find_dollar(text):#Finding the dollar sign in the text
    line=re.sub(r'\$\d+(?:\.\d+)?','PRICE',text)
    return "".join(line)

def replace_emoji(text):
    emoji_pattern = re.compile("["
    u"\U0001F600-\U0001F64F" # emoticons
    u"\U0001F300-\U0001F5FF" # symbols & pictographs
    u"\U0001F680-\U0001F6FF" # transport & map symbols
    u"\U0001F1E0-\U0001F1FF" # flags (iOS)
    u"\U00002702-\U000027B0"
    u"\U000024C2-\U0001F251"
    "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'EMOJI', text) 

puncts = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']',
          '>', '%', '=', '#', '*', '+', '\\', '•', '~', '@', '£', '·', '_', '{', '}', '©', '^',
          '®', '`', '<', '→', '°', '€', '™', '›', '♥', '←', '×', '§', '″', '′', 'Â', '█',
          '½', 'à', '…', '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶',
          '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', '▒', '：', '¼',
          '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲',
          'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', '∙', '）', '↓', '、', '│', '（', '»', '，', '♪',
          '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√']

def clean_text(text: str) -> str:
    text = str(text)
    for punct in puncts + list(string.punctuation):
        if punct in text:
            text = text.replace(punct, f'')
    return text
   
def replace_asterisk(text):
    text = re.sub("\*", 'ABUSE ', text)
    return text

def remove_duplicates(text):
    text = re.sub(r'\b(\w+\s*)\1{1,}', '\\1', text)
    return text

def change(text):
    if(text == ''):
        return text
  #calling the subfunctions in the cleaning function
    text = replace_email(text)
    text = replace_url(text)
    text = unique_char(rep,text)
    text = replace_asterisk(text)
    text = remove_duplicates(text)
    text = clean_text(text)
    return text

In [ ]:
import pandas as pd

# Loading different csv files for cleaning 
pathname = "/content/drive/My Drive/IBM_Hackathon_2020/Final_Datasets/Data.csv"
# pathname = "/content/drive/My Drive/IBM_Hackathon_2020/Final_Datasets/Train_India.csv"
# pathname = "/content/drive/My Drive/IBM_Hackathon_2020/Final_Datasets/Test_Data.csv"
# pathname = "/content/drive/My Drive/IBM_Hackathon_2020/Final_Datasets/Test_India.csv"
Data = pd.read_csv(pathname) #reading the file
Data['full_text'] = Data['full_text'].apply(lambda x : change(x)) # Running cleaning and normalization function on datasets
Data.to_csv(pathname) #converting to CSV 